In [16]:
import tensorflow as tf
import tensorflow as tf
import numpy as np

from turtle import up
import pandas as pd
import scipy
from scipy.stats import iqr


def windowed_dataset(dataset: tf.data.Dataset, window_size=5, shift=1, stride=1):
    if window_size == None or window_size <= 0:
        return dataset

    def sub_to_batch(t1, t2=None):
        t1_batches = t1.batch(window_size, drop_remainder=True)

        if t2 is not None:
            t2_batches = t2.batch(window_size, drop_remainder=True)
            return tf.data.Dataset.zip(t1_batches, t2_batches)
        else:
            return t1_batches

    def pick_batch(t1, t2: tf.Tensor | None):
        #return (t1, t2)
        if t2 is None:
            return t1
        else:
            print(t2.get_shape())
            shape = t2.get_shape()
            l = len(shape)
            print(l)
            newshape = []
            for i in range(l):
                print(i)
                if i + 2 == l:
                    newshape.append(shape[i] - 1)
                else:
                    newshape.append(shape[i])
            print(newshape)
            t2_batches = t2[]
            return (t1, t2_batches)

    return dataset.window(window_size, shift=shift, stride=stride, drop_remainder=True).flat_map(sub_to_batch).batch(window_size, drop_remainder=True).map(pick_batch)
    return windows  #.batch(window_size, drop_remainder=drop_remainder)


def windowed_batch_dataframe(df: pd.DataFrame, window_size=4, batch_size=None):
    df = df.copy()
    if batch_size is not None and batch_size > 0:
        dataset = windowed_dataset(tf.data.Dataset.from_tensor_slices(df.values), window_size).batch(batch_size=batch_size, drop_remainder=True)
    else:
        dataset = windowed_dataset(tf.data.Dataset.from_tensor_slices(df.values), window_size)
    return dataset


def windowed_batch_dataset(ds: tf.data.Dataset, window_size=4, batch_size=None):

    if batch_size is not None and batch_size > 0:
        dataset = windowed_dataset(ds, window_size).batch(batch_size=batch_size, drop_remainder=True)
    else:
        dataset = windowed_dataset(ds, window_size)
    return dataset


def convert_to_numpy(ds: tf.data.Dataset):
    # 将 dataset 转换为 NumPy 数组
    numpy_array = None
    for data in ds.as_numpy_iterator():
        # 确保数据是 NumPy 数组
        data_np = np.array(data)

        # 如果 numpy_array 还没有初始化，初始化它
        if numpy_array is None:
            numpy_array = data_np
        else:
            # # 确保数据的形状与 numpy_array 相同
            # if data_np.shape[1:] != numpy_array.shape[1:]:
            #     raise ValueError("All data items must have the same shape except for the first dimension.")

            # 垂直堆叠数据
            numpy_array = np.vstack((numpy_array, data_np))

    return numpy_array


def get_shape(*args):
    shapes = []
    for arg in args:
        shapes.append(arg.shape if arg is not None else None)
    return shapes

In [17]:
rows = 20
feature_cols = ['a', 'b']
label_cols = ['label']
columns = feature_cols + label_cols  #['a', 'b', 'c']
cols = len(columns)
data_x = np.array([[i+10 for c in np.arange(1, cols)] + [i ] for i in np.arange(1, rows + 1)])

df = pd.DataFrame(data_x, columns=columns)

print(df)


     a   b  label
0   11  11      1
1   12  12      2
2   13  13      3
3   14  14      4
4   15  15      5
5   16  16      6
6   17  17      7
7   18  18      8
8   19  19      9
9   20  20     10
10  21  21     11
11  22  22     12
12  23  23     13
13  24  24     14
14  25  25     15
15  26  26     16
16  27  27     17
17  28  28     18
18  29  29     19
19  30  30     20


In [18]:
win_size =2
batch_size = 0


def splice(a, b):
    print('---------')
    print(a)
    return (a, b)


ds = tf.data.Dataset.from_tensor_slices((df[feature_cols], df[label_cols]))
x = windowed_batch_dataset(ds, win_size, batch_size).map(splice)

for aa,bb in x:
    print('--')
    print(bb)
    
# print('x1 ===')
# print(y)
# print('===')
# print(y)

# a, b = convert_to_numpy(x), convert_to_numpy(y)
# print(get_shape(a))
# print(a)

(2, 2, 1)
3
0
1
2
[2, 1, 1]


ValueError: in user code:

    File "C:\Users\tangj15\AppData\Local\Temp\1\ipykernel_25336\2259757678.py", line 41, in pick_batch  *
        t2_batches = t2[newshape]

    ValueError: slice index 2 of dimension 0 out of bounds. for '{{node strided_slice}} = StridedSlice[Index=DT_INT32, T=DT_INT32, begin_mask=0, ellipsis_mask=0, end_mask=0, new_axis_mask=0, shrink_axis_mask=7](args_1, strided_slice/stack, strided_slice/stack_1, strided_slice/stack_2)' with input shapes: [2,2,1], [3], [3], [3] and with computed input tensors: input[1] = <2 1 1>, input[2] = <3 2 2>, input[3] = <1 1 1>.
